In [26]:
import pandas as pd
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import seaborn as sns

In [3]:
df = pd.read_parquet(r"/Users/loufourneaux/Desktop/EPFL/MA1/ML/project2/All_Relative_Results_Cleaned.parquet")

In [4]:
df_clean = df.dropna()
index = df_clean.columns.get_loc('time(s)')
df_right = df_clean.iloc[:, index+1:]
X = df_right
Y = df_clean['Exercise']

In [5]:
label_encoder = LabelEncoder()
Y_encoded = label_encoder.fit_transform(Y)

In [6]:
X_tensor = torch.tensor(X.values, dtype=torch.float32)
y_tensor = torch.tensor(Y_encoded, dtype=torch.long)

In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(X_tensor, y_tensor, test_size=0.2)

train_dataset = TensorDataset(X_train,Y_train)
test_dataset = TensorDataset(X_test,Y_test)
trainLoader = DataLoader(train_dataset, batch_size=32, shuffle=True)
testLoader = DataLoader(test_dataset, batch_size=32 , shuffle=True)

In [23]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNetwork, self).__init__()
        self.layer1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(hidden_size, output_size)

        self.reset_parameters()

    def forward(self, x):
        x = self.layer1(x)
        x = self.relu(x)
        x = self.layer2(x)
        return x
        
    def reset_parameters(self):
        for layer in self.children():
            if hasattr(layer, 'reset_parameters'):
                layer.reset_parameters()

In [9]:
class GRUNeuralNetwork(nn.Module):
    def __init__(self):
        super(GRUNeuralNetwork, self).__init__()
        self.gru = nn.GRU(X_train.shape[1], 64)#, batch_first=True)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(64, 7)

    def forward(self, x):
        # x est de la forme (batch_size, sequence_length, input_size)
        output, _ = self.gru(x)
        # Prenez seulement la sortie de la dernière étape de la séquence
        #output = output[:, -1, :]
        output = self.relu(output)
        output = self.layer2(output)
        return output

**cross-validation**

In [31]:
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from skorch import NeuralNetClassifier

In [33]:
# Définissez le nombre de plis
num_folds = 5
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

In [39]:
# Instanciez votre modèle avec meilleurs parametres 
model = NeuralNetwork(X_train.shape[1], 64, 7)

In [40]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [ ]:
# Boucle sur les plis
for fold, (train_index, val_index) in enumerate(kf.split(X_train)):
    # Créez des ensembles d'entraînement et de validation pour ce pli
    X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
    y_train_fold, y_val_fold = Y_train[train_index], Y_train[val_index]

    # Créez des DataLoader pour l'entraînement et la validation
    train_dataset = TensorDataset(X_train_fold, y_train_fold)
    val_dataset = TensorDataset(X_val_fold, y_val_fold)
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

    # Réinitialisez les paramètres du modèle avant chaque pli
    model.reset_parameters()

    # Entraînez votre modèle
    num_epochs = 10  # ou choisissez le nombre d'époques approprié
    for epoch in range(num_epochs):
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    # Évaluez votre modèle sur l'ensemble de validation
    model.eval()
    total_test = 0
    correct_test = 0

    y_true = []
    y_pred = []
    with torch.no_grad():
        for inputs, labels in val_loader:
            labels=labels.long()
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            y_true.extend(labels.tolist())
            y_pred.extend(predicted.tolist())
            total_test += labels.size(0)
            correct_test += (predicted == labels).sum().item()

            # Calcul des métriques d'évaluation (précision, rappel, etc.)

    test_accuracy = 100 * correct_test / total_test
    print(f'Exactitude sur l\'ensemble {fold} de test: {test_accuracy}%')       


**hyper parmeters tuning**

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score
from torch.utils.data import DataLoader, TensorDataset

def tune_hyperparameters(X_train, y_train):
    param_grid = {
        'hidden_size': [32, 64, 128],
        'lr': [0.001, 0.01, 0.1]
    }

    model = NeuralNetwork(input_size=X_train.shape[1], output_size=num_classes)

    train_dataset = TensorDataset(X_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

    # Convert the PyTorch model to a skorch classifier
    classifier = NeuralNetClassifier(
        model,
        criterion=nn.CrossEntropyLoss,
        optimizer=optim.Adam,
        max_epochs=10,  # or choose an appropriate number of epochs
        iterator_train=train_loader
    )

    # Use GridSearchCV for hyperparameter tuning
    grid_search = GridSearchCV(classifier, param_grid, scoring='accuracy', cv=3)
    grid_search.fit(X_train, y_train)

    # Get the best hyperparameters
    best_hyperparams = grid_search.best_params_

    return best_hyperparams
